In [2]:
from moabb.datasets import Liu2024
from mne import events_from_annotations
from mne import Epochs
import numpy as np
from fbcsp import MLEngine

from sklearn.model_selection import train_test_split


dataset = Liu2024()
data = dataset.get_data()



c:\Users\abrah\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\datasets\liu2024.py:251: RuntimeWarning: The unit for channel(s) STI has changed from NA to V.
  raw.set_channel_types(mapping)
c:\Users\abrah\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\datasets\liu2024.py:251: RuntimeWarning: The unit for channel(s) STI has changed from NA to V.
  raw.set_channel_types(mapping)
c:\Users\abrah\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\datasets\liu2024.py:251: RuntimeWarning: The unit for channel(s) STI has changed from NA to V.
  raw.set_channel_types(mapping)
c:\Users\abrah\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\datasets\liu2024.py:251: RuntimeWarning: The unit for channel(s) STI has changed from NA to V.
  raw.set_channel_types(mapping)
c:\Users\abrah\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\datasets\liu2024.py:251: RuntimeWarning: The unit for channel(s) STI has changed from NA to V

In [22]:
''' Function ot obtain the raw epochs from a mne.io.Raw object '''
def get_data_from_run(raw_run):
    events_id = {'left_hand': 0, 'right_hand': 1}
    events, event_id = events_from_annotations(raw_run, event_id=events_id)
    epochs = Epochs(raw_run, events, event_id=event_id, tmin=0.0, tmax=4.0, preload=True, baseline=None, verbose=0)
    X = epochs.get_data(picks=['eeg'])
    y = epochs.events[:,2]

    return X, y

In [28]:
res = dict()


for subject in data:
    for session in data[subject]:
        for run in data[subject][session].values():
            X, y = get_data_from_run(run)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
    model = MLEngine(m_filters=2, fs = 500)
    results = model.experiment(X_train, y_train, X_test, y_test)
    res[subject] = (results['train_acc'], results['test_acc'])

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.25
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.75
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.375
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.5
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.375
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 0.90625

Testing Accuracy = 0.25
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.25
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.5
 

Used Annotations descriptions: ['left_hand', 'right_hand']
Training Accuracy = 1.0

Testing Accuracy = 0.375

In [30]:
print('accuracy of each subject')
print('subject, training accuracy, testing accuracy')
for key, value in dict(sorted(res.items())).items():
    print(f"{str(key)}, {round(value[0],3)}, {round(value[1],3)}")

accuracy of each subject
subject, training accuracy, testing accuracy
1, 1.0, 0.25
2, 1.0, 0.75
3, 1.0, 0.375
4, 1.0, 0.5
5, 1.0, 0.375
6, 0.906, 0.25
7, 1.0, 0.25
8, 1.0, 0.5
9, 1.0, 0.375
10, 1.0, 0.625
11, 1.0, 0.375
12, 1.0, 0.75
13, 0.75, 0.5
14, 1.0, 0.5
15, 1.0, 0.375
16, 0.969, 0.25
17, 1.0, 0.375
18, 1.0, 0.5
19, 0.969, 0.25
20, 0.875, 0.25
21, 1.0, 0.25
22, 1.0, 0.375
23, 0.969, 0.25
24, 1.0, 0.5
25, 1.0, 0.625
26, 0.969, 0.25
27, 0.938, 0.375
28, 1.0, 0.375
29, 0.906, 0.5
30, 1.0, 0.625
31, 1.0, 0.375
32, 0.969, 0.625
33, 1.0, 0.375
34, 1.0, 0.5
35, 1.0, 0.125
36, 0.969, 0.125
37, 0.969, 0.625
38, 1.0, 0.5
39, 1.0, 0.375
40, 1.0, 0.5
41, 1.0, 0.25
42, 1.0, 0.75
43, 1.0, 0.25
44, 1.0, 0.25
45, 1.0, 0.25
46, 1.0, 0.5
47, 1.0, 0.375
48, 1.0, 0.375
49, 1.0, 0.375
50, 1.0, 0.375


In [ ]:






for subject in dataset.subject_list:
    raw_subject = data[subject]
    
    for i, session in enumerate(raw_subject):

        ''' Since the session of a subject has 6 runs, here we join the 6 runs in a single X (total epochs = 48*6)'''
        for j, run in enumerate(raw_subject[session].values()):
            X_run, y_run = get_data_from_run(run)
            X[48*j:48*(j+1), :, :] = X_run
            y[48*j:48*(j+1)] = y_run


        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
        model = MLEngine(m_filters=2, fs = 250)
        results = model.experiment(X_train, y_train, X_test, y_test)
        res[subject + (9*i)] = (results['train_acc'], results['test_acc'])


print('accuracy of each subject')
print('subject, training accuracy, testing accuracy')
for key, value in dict(sorted(res.items())).items():
    print(f"{str(key)}, {round(value[0],3)}, {round(value[1],3)}")
 
